To Do:
1. Download a HTML page with links to songs

2. Extract hyperlinks of song pages

3. Download and extract the song lyrics

4. Vectorize the text using the Bag Of Words method

5. train a classification model that predicts the artist from a piece of text

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# Class to read, extract hyperlinks and prepre data in dataframe format

In [4]:
class lyric:
    def artist(url):
        soup = BeautifulSoup(requests.get(url).text,'html.parser')
        table = soup.find('table', {'class':'tdata'})
        href_links = []
        lyric_links=[]
        lyrics=[] 
        for link in table.findAll('a'):
            href_links.append(link.get('href'))

        for i in href_links:
            if ([i.startswith('/sublyric/'),i.startswith('/lyric-lf/')]):
                i = 'https://www.lyrics.com' + i
                lyric_links.append(i)

        for t in lyric_links:
            i = (BeautifulSoup(requests.get(t).text,'html.parser').find(id='lyric-body-text')).get_text()
            lyrics.append(i)
        df = pd.DataFrame(lyrics)
        df.rename(columns={0: 'lyric'}, inplace=True)
        df.drop_duplicates(subset ='lyric',keep = 'first' , inplace = True, ignore_index=True)
        df['lyric']=df['lyric'].str.replace('\W', ' ')
        return df
        

# Download Billie Rogue's lyrics

In [5]:
Billie_Rogue = lyric
Billie_Rogue = Billie_Rogue.artist('https://www.lyrics.com/artist/Billie-Rogue/2137945805')

/var/folders/66/m2k6yl2s06s1fcd6nnnnf_f40000gn/T/ipykernel_2057/2388873687.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  df['lyric']=df['lyric'].str.replace('\W', ' ')


# Download K_Ray's lyrics

In [6]:
K_Ray = lyric
K_Ray = K_Ray.artist('https://www.lyrics.com/artist/K.-Ray/2137906147')

/var/folders/66/m2k6yl2s06s1fcd6nnnnf_f40000gn/T/ipykernel_2057/2388873687.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  df['lyric']=df['lyric'].str.replace('\W', ' ')


# Encode labels

In [7]:
Billie_Rogue['Artist'] = 0
K_Ray['Artist'] = 1

In [8]:
frames = [Billie_Rogue, K_Ray]

df = pd.concat(frames, ignore_index=True)

# Vectorize the texts

In [9]:
vectorizer_tf = TfidfVectorizer(stop_words='english')
Y = vectorizer_tf.fit_transform(df['lyric'])
df_tfidf = pd.DataFrame(Y.toarray(),columns=vectorizer_tf.get_feature_names())

# Decomposition using NMF

In [10]:
model = NMF(n_components=15, init='random', random_state=0)
W = model.fit_transform(df_tfidf)
H = model.components_

/Users/Disalo/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [11]:
components_df = pd.DataFrame(model.components_, columns=vectorizer_tf.get_feature_names())
components_df

,10,11,12,14,17,23,25,40,420,45,...,yard,yeah,year,years,yelling,yes,yo,younger,zimmerman,zones
0,0.000000,0.000000,0.267234,0.000000e+00,0.000000e+00,0.000000e+00,0.121433,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.008065,0.083673,0.000000e+00,0.000000,0.000000e+00,0.157736,0.000000,0.183484,0.000000e+00
1,0.000000,0.000000,0.000008,0.000000e+00,0.000000e+00,0.000000e+00,0.000349,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.123843,0.089499,0.000000e+00,0.000000,0.000000e+00,0.001051,0.000000,0.000000,0.000000e+00
2,0.000000,0.000232,0.000083,0.000000e+00,0.000000e+00,0.000000e+00,0.003544,0.000000e+00,0.000000e+00,0.000465,...,0.000000e+00,0.186827,0.002255,0.000000e+00,0.000232,0.000000e+00,0.004133,0.375462,0.000000,0.000000e+00
3,0.000000,0.000000,0.000240,0.000000e+00,0.000000e+00,0.000000e+00,0.010295,0.000000e+00,0.000000e+00,0.000000,...,7.403418e-11,0.193422,0.011906,0.000000e+00,0.000000,1.480684e-10,0.472843,0.000000,0.000000,0.000000e+00
4,0.000000,0.000066,0.000000,0.000000e+00,0.000000e+00,5.383219e-09,0.000000,1.508182e-09,8.329147e-11,0.000132,...,0.000000e+00,0.000000,0.000000,1.508182e-09,0.000066,0.000000e+00,0.000000,0.000000,0.000108,0.000000e+00
5,0.000000,0.000000,0.000053,0.000000e+00,0.000000e+00,0.000000e+00,0.002275,0.000000e+00,2.607438e-01,0.000000,...,1.351765e-09,1.683842,0.187631,0.000000e+00,0.000000,2.703531e-09,0.298431,0.000000,0.000000,0.000000e+00
6,0.000000,0.000000,0.000062,0.000000e+00,3.276233e-01,0.000000e+00,0.002645,0.000000e+00,0.000000e+00,0.000000,...,5.307622e-11,0.325504,0.002346,0.000000e+00,0.000000,1.061524e-10,0.376267,0.000000,0.000000,3.276233e-01
7,0.000000,0.000000,0.000103,0.000000e+00,0.000000e+00,1.118191e+00,0.004428,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.000316,0.003072,0.000000e+00,0.000000,0.000000e+00,0.005839,0.000000,0.000000,0.000000e+00
8,0.000000,0.000000,0.000023,2.595260e-10,0.000000e+00,0.000000e+00,0.001006,0.000000e+00,0.000000e+00,0.000000,...,2.202115e-10,0.000078,0.000688,0.000000e+00,0.000000,4.404230e-10,0.001301,0.000000,0.000000,0.000000e+00
9,0.232251,0.000000,0.000290,0.000000e+00,0.000000e+00,0.000000e+00,0.012446,0.000000e+00,0.000000e+00,0.000000,...,9.538768e-09,0.347413,0.024065,0.000000e+00,0.000000,1.907754e-08,0.735293,0.000000,0.000000,0.000000e+00


# Extract words with highest value

In [13]:
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    print(f'For topic {topic+1} the words with the highest value are:')
    print(tmp.nlargest(3))
    print('\n')

For topic 1 the words with the highest value are:
black      1.003309
wishing    0.993740
running    0.910263
Name: 0, dtype: float64


For topic 2 the words with the highest value are:
em      0.491900
days    0.374385
ways    0.374385
Name: 1, dtype: float64


For topic 3 the words with the highest value are:
head     1.764066
trust    1.610074
tired    1.501848
Name: 2, dtype: float64


For topic 4 the words with the highest value are:
ignore    1.569144
story     1.569144
night     1.382533
Name: 3, dtype: float64


For topic 5 the words with the highest value are:
sow         2.039814
lulladie    1.019907
eyes        0.890565
Name: 4, dtype: float64


For topic 6 the words with the highest value are:
smoke    3.911156
want     2.053348
yeah     1.683842
Name: 5, dtype: float64


For topic 7 the words with the highest value are:
mind     1.026058
kyrie    0.655247
post     0.655247
Name: 6, dtype: float64


For topic 8 the words with the highest value are:
learned    1.397739
23   

# Test the model

In [133]:
new_sample = """We shooting dice in the section where the Bacardi at
Now the clerk mad at us yelling hurry up and buy """
 
# Transform the TF-IDF
X_new = vectorizer_tf.transform([new_sample])

# Transform the TF-IDF: nmf_features
nmf_features = model.transform(X_new)
 
pd.DataFrame(nmf_features)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,0.0,0.0,0.005985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038132,0.0
